<a href="https://colab.research.google.com/github/seyeonlim/Avy-Savvy-Skiing/blob/main/DL_Pathology_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# prompt: import csv file in my google drive
import pandas as pd

measurements = pd.read_csv('/content/gdrive/Shareddrives/DL_Pathology_Detection_Special_Studies/Measurements.csv')

identifiers = [
    "Aithal_2013",
    "Aithal_2014",
    "Aithal_2014b",
    "Aithal_2015",
    "Aithal_2017a",
    "Aithal_2017b",
    "Aithal_2022",
    "AlMakadma_2021",
    "Ellison_2012",
    "Hunter_2016",
    "Merchant_2010",
    "Merchant_2021",
    "Myers_2018",
    "Pitaro_2016",
    "Sanford_2009",
    "Sanford_2014",
    "Voss_2016",
    "Werner_2010"
]

print(measurements.head())


  Identifier  SubjectNumber  Session   Ear Instrument   Age AgeCategory  \
0  Abur_2014              1        1  Left     HearID  20.0       Adult   
1  Abur_2014              1        1  Left     HearID  20.0       Adult   
2  Abur_2014              1        1  Left     HearID  20.0       Adult   
3  Abur_2014              1        1  Left     HearID  20.0       Adult   
4  Abur_2014              1        1  Left     HearID  20.0       Adult   

  EarStatus  TPP  AreaCanal  PressureCanal SweepDirection  Frequency  \
0    Normal -5.0   0.000044              0        Ambient    210.938   
1    Normal -5.0   0.000044              0        Ambient    234.375   
2    Normal -5.0   0.000044              0        Ambient    257.812   
3    Normal -5.0   0.000044              0        Ambient    281.250   
4    Normal -5.0   0.000044              0        Ambient    304.688   

   Absorbance         Zmag      Zang  
0    0.033338  113780000.0 -0.233504  
1    0.031571  103585000.0 -0.235778  

In [ ]:
import matplotlib.pyplot as plt

def create_scatterplot(x, y, xlabel, ylabel, title, colors, ylog = False):
  # axis goes from 500 to 6000
  """
  Creates a scatter plot.

  Args:
    x: The x-axis data.
    y: The y-axis data.
    xlabel: The label for the x-axis.
    ylabel: The label for the y-axis.
    title: The title of the plot.
  """
  fig = plt.figure()
  plt.scatter(x, y, s = 1, c = colors)
  plt.xlabel(xlabel)
  plt.xscale('log')  # Set x-axis to log scale
  if ylog:
    plt.yscale('log') # set y to log scale
  plt.ylabel(ylabel)
  plt.title(title)
  #plt.show()
  return fig

#returns log of freq
def get_freq(pi_identifier, data):
  return data[data['Identifier'] == pi_identifier]['Frequency']

def get_absorbance(pi_identifier, data):
  return data[data['Identifier'] == pi_identifier]['Absorbance']

#returns log of mag
def get_zmag(pi_identifier, data):
  return data[data['Identifier'] == pi_identifier]['Zmag']

def get_zang(pi_identifier, data):
  return data[data['Identifier'] == pi_identifier]['Zang']

def get_leftright_colors(pi_identifier, data):
  return data[data['Identifier'] == pi_identifier]['Ear'].map({'Left':'tab:blue', 'Right':'tab:red'})

# Get the first row's' instrument - But does each publication use the same instrument for all measurements?
def get_instrument(pi_identifier, data):
  return data[data['Identifier'] == pi_identifier]['Instrument'].iloc[0]

#cyan for normal, black for all others
def get_ear_status_colors(pi_identifier, data):
  ear_status = data[data['Identifier'] == pi_identifier]['EarStatus']
  colors = ['cyan' if category == 'Normal' else 'black' for category in ear_status]
  return colors

In [ ]:
np.unique(get_ear_status_colors("Aithal_2014b", measurements))

array(['tab:cyan', 'tab:red'], dtype=object)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

with PdfPages('baby-wai-graphs.pdf') as pdf:
  for identifier in identifiers:
    # get frequency
    freq = get_freq(identifier, measurements)

    #leftright = get_leftright_colors(identifier, measurements)
    ear_status_colors = get_ear_status_colors(identifier, measurements)
    # create scatter for absorbance
    absorbance = get_absorbance(identifier, measurements)
    instrument = get_instrument(identifier, measurements)

    fig = create_scatterplot(freq, absorbance, "frequency", "absorbance", identifier + " frequency vs. absorbance" + " (" + str(instrument) + ")", ear_status_colors)
    pdf.savefig(fig)  # Save the current figure to the PDF
    plt.close()  # Close the figure

    #zmag
    zmag = get_zmag(identifier, measurements)
    if not np.isnan(zmag.iloc[0]):
      fig = create_scatterplot(freq, zmag, "frequency", "Magnitude", identifier + " frequency vs. magnitude" + " (" + str(instrument) + ")", ear_status_colors, ylog=True)
      pdf.savefig(fig)  # Save the current figure to the PDF
      plt.close()  # Close the figure

    #zang
    zang = get_zang(identifier, measurements)
    if not np.isnan(zang.iloc[0]):
      fig = create_scatterplot(freq, zang, "frequency", "Angle", identifier + " frequency vs. angle" + " (" + str(instrument) + ")", ear_status_colors)
      pdf.savefig(fig)  # Save the current figure to the PDF
      plt.close()  # Close the figure


**Identifiers for publications with Infant measurements**

Aithal_2013
Aithal_2014
Aithal_2014b
Aithal_2015
Aithal_2017a
Aithal_2017b
Aithal_2022
AlMakadma_2021
Ellison_2012
Hunter_2016
Merchant_2010
Merchant_2021
Myers_2018
Pitaro_2016
Sanford_2009
Voss_2016
Werner_2010

cyan, black for easy seeing, color code normal, abnormal
log log scale for magnitude v freq
freq is always log scale

use aithal, ellison, sanford, using only absorbance
- may not have same frequencies, could do interpolation
- do one training w/o modifying dataset - sanford and esllison
- ((f,a), {n,a}) where f, a are vectors
- confusion matrix, possible for showing accuracy


Test Change

After you have made changes to your notebook, you can commit and push them to the repository. To do so from within a Colab notebook, click File → Save a copy in GitHub. You will be prompted to add a commit message, and after you click OK, the notebook will be pushed to your repository.